## Introduction

We will use webscraping to find out the capture rate for each pokemon.

We will use serebii.net to find the these values.

In [3]:
import requests
from bs4 import BeautifulSoup
import lxml

In [4]:
urls = []

for i in range(721):
    num = i + 1
    num = str(num)
    if len(num) != 3:
        diff = 3 - len(num) 
        num = diff * "0" + num  # padding
    link = 'https://serebii.net/pokedex-xy/' + num + '.shtml'
    urls.append(link)
    

In [22]:
# along with the capture rates, we should probably find the names too
names = []
rates = []
breed_info_list = []
for page in urls:
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'lxml')
    # find all tables in the webpage
    items = soup.find_all('table', class_ = 'dextable')
    # find name
    rate = items[1].find_all("td",class_="fooinfo")[7].text # look at 2nd table and the 8th item
    rates.append(rate)
    # find capture rate
    name = items[1].find_all("td",class_="fooinfo")[0].text # look at 2nd table and the 2nd item
    names.append(name)
    # find breed info

    breed_info = items[4].find("form") # look at the 5th table
    if breed_info:  # if the breed info exists, append 1
        breed_info_list.append(1)
    else:
        breed_info_list.append(0) 

In [61]:
import pandas as pd

df = pd.DataFrame({"names":names,
                   "capture_rate":rates,
                   "can_breed":breed_info_list})

df["capture_rate_clean"] = df["capture_rate"].str.extract("(\d+)")

# we make the pokemon number ourselves
df = df.reset_index()
df['pkmn_number'] = df['index'] + 1

df.to_csv("pkmn_capture_rates.csv", index = False)

In [63]:
df.drop("index", axis=1).to_csv("pkmn_capture_rates.csv", index = False)